# Computational Intelligence Coursework - Ali

## Imports, Functions, Model, Data Loading

### Imports & Seed

In [1]:
# general imports
import numpy as np

# torch & data manipulation imports
import torch
from torch.utils.data import ConcatDataset, Subset, DataLoader
import torchvision
import torchvision.transforms as transforms

# model-related imports
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

# seed for reproducibility
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

/Users/alishihab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Define Custom Preprocessing Functions

In [2]:
from torch.utils.data import Dataset

# DATA PREPROCESSING
class CustomDataset(Dataset):
    def __init__(self, dataset, indices, transform=None):
        self.dataset = dataset
        self.indices = indices
        self.transform = transform

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        image, label = self.dataset[self.indices[idx]]
        if self.transform:
            image = self.transform(image)
        return image, label

# calculate mean & standard deviation based on dataset
def calc_mean_std(dataset):
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=2)
    mean_sum = 0.
    var_sum = 0.
    total_images_count = 0
    for images, _ in dataloader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean_sum += images.mean(2).sum(0)
        var_sum += images.var(2).sum(0)
        total_images_count += batch_samples

    mean = mean_sum / total_images_count
    var = var_sum / total_images_count
    std = np.sqrt(var)

    return mean, std

# define transformations for data augmentation
def train_transform(data, mean, std):
  data = normalize(data, mean, std)
  transform = transforms.Compose([
                                transforms.RandomHorizontalFlip(0.25),
                                transforms.RandomVerticalFlip(0.25),
                                transforms.RandomGrayscale(0.25),
                                transforms.RandomCrop(32, padding=4)
                                 ])

  fred_transform = transforms.Compose([

                                transforms.RandomHorizontalFlip(),
                                transforms.RandomRotation(10),
                                torchvision.transforms.RandomCrop(
                                    32, padding=4, padding_mode='reflect'),
                            ])
  return fred_transform(data)

# define normalisation
def normalize(data, mean, std):
  transform = transforms.Compose([
                                transforms.Normalize(mean, std)
                                ])
  return transform(data)

### Define Model Architecture

#### Ali's Net

In [21]:
class Net(nn.Module):
    def __init__(self, dropout_prob):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 32, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(32 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 32 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

    def freeze_all_but_last():
        for name, param in self.named_parameters():
            if 'fc2' not in name:
                param.requires_grad = False

    # Extract weights from the last layer
    def extract_weights():
        return [p.data.numpy() for p in self.fc2.parameters()]

#### Fred's Net

In [20]:
class Net2(nn.Module):
    def __init__(self, num_classes=10):
        super(Net2, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm2d(64)

        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.batch_norm2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.batch_norm3 = nn.BatchNorm2d(128)
        self.max_pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.1)

        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.batch_norm4 = nn.BatchNorm2d(128)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.relu5 = nn.ReLU()
        self.batch_norm5 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 256)
        self.dropout2 = nn.Dropout(0.1)
        self.fc3 = nn.Linear(256, num_classes)

    def forward(self, x):
      #print("input", x.shape)
      x = self.conv1(x)
      x = self.relu1(x)
      x = self.batch_norm1(x)
      #print("conv1",x.shape)
      x = self.conv2(x)
      x = self.relu2(x)
      x = self.batch_norm2(x)
      #print("conv2",x.shape)
      x = self.conv3(x)
      x = self.relu3(x)
      x = self.batch_norm3(x)
      x = self.max_pool1(x)
      x = self.dropout1(x)
      #print("conv3",x.shape)
      x = self.conv4(x)
      x = self.relu4(x)
      x = self.batch_norm4(x)
      x = self.max_pool2(x)
     #print("conv4",x.shape)
      x = self.flatten(x)
      #print("flatten",x.shape)
      x = self.fc1(x)
      x = self.relu5(x)
      x = self.batch_norm5(x)
      #print("fc1",x.shape)
      x = self.fc2(x)
      #print("fc2",x.shape)

      x = self.dropout2(x)

      x = self.fc3(x)
      #print("fc3",x.shape)
      return x

    def freeze_all_but_last():
        for name, param in self.named_parameters():
            if 'fc3' not in name:
                param.requires_grad = False

    # Extract weights from the last layer
    def extract_weights():
        return [p.data.numpy() for p in self.fc3.parameters()]

### Checkpointing

In [4]:
def save_checkpoint(state, filename="checkpoint.pth.tar"):
    torch.save(state, filename)

### Data Loading & Preparation

In [5]:
# load CIFAR-10 dataset & convert to tensor
train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True,
                                         transform=transforms.ToTensor())

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True,
                                        transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# dataset hyperparameters
num_folds = 5
test_size = 0.20

# combine train and test datasets for stratified splitting
combined_set = ConcatDataset([train_set, test_set])

# STRATIFIED SPLIT
# collect the labels
labels = [y for _, y in combined_set]

# stratified split subset indices
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
train_idx, test_idx = next(sss.split(np.zeros(len(labels)), labels))

# subset using indices & collect associated labels
stratified_train_set, stratified_train_labels = Subset(combined_set, train_idx).dataset, [labels[i] for i in range(len(labels)) if i in train_idx]
stratified_test_set, stratified_test_labels = Subset(combined_set, test_idx).dataset, [labels[i] for i in range(len(labels)) if i in test_idx]

# create StratifiedKFold object for train set only
skf = StratifiedKFold(n_splits=num_folds)

## Baselines

### Gradient Based - Adam - Using Ali's Net

In [7]:
# function for training and evaluating the model
def adam_train_and_validate(model, train_loader, test_loader, criterion, optimizer, mean, std, epochs=30):
    model.to(device)
    model.train()

    # early stopping parameters
    early_stopping_patience = 5  # number of epochs to wait for improvement before stopping
    early_stopping_counter = 0    # counter for epochs without improvement
    best_accuracy = 0             # track the best accuracy

    # train
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        i = 0

        for inputs, train_load_labels in train_loader:
            inputs = train_transform(inputs, mean, std)
            inputs, train_load_labels = inputs.to(device), train_load_labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, train_load_labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += train_load_labels.size(0)
            correct += (predicted == train_load_labels).sum().item()
            i += 1
            if i % 20 == 0:
              batch_accuracy = 100 * correct / total
              print(f'{i}th Batch Loss: {loss.item():.4f} Batch Accuracy: {batch_accuracy:.4f}')

        epoch_loss = running_loss / total
        epoch_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{epochs}] Loss: {loss.item():.4f} Epoch Accuracy: {epoch_accuracy:.4f}')

        model.eval()
        correct = 0
        total = 0

        # validate
        with torch.no_grad():
            for inputs, test_load_labels in test_loader:
                inputs = normalize(inputs, mean, std)
                inputs, test_load_labels = inputs.to(device), test_load_labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += test_load_labels.size(0)
                correct += (predicted == test_load_labels).sum().item()

        validation_accuracy = 100 * correct / total

        # check if the current validation accuracy is better than the best recorded accuracy
        if validation_accuracy > best_accuracy:
            best_accuracy = validation_accuracy
            early_stopping_counter = 0  # Reset the counter
            # save the model checkpoint
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, filename=f"best_model_epoch_{epoch+1}.pth.tar")
        else:
            early_stopping_counter += 1

        print(early_stopping_counter)
        # check if early stopping should be triggered
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break

        print(f'Validation Accuracy: {validation_accuracy:.2f}%')
    return model

# function for testing the model
def test(model, test_loader, mean, std):

    model.to(device)
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, test_load_labels in test_loader:
            inputs = normalize(inputs, mean, std)
            inputs, test_load_labels = inputs.to(device), test_load_labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += test_load_labels.size(0)
            correct += (predicted == test_load_labels).sum().item()


    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')
    return model

In [8]:
# Model hyperparameters
dropout_prob = 0.30
num_epochs = 20
batch_size = 128

model = None

# call function
# Main loop for k-fold cross-validation
for fold, (train_fold_indices, val_fold_indices) in enumerate(skf.split(train_idx, stratified_train_labels)):
    print(f'Fold {fold + 1}/{num_folds}')
    mean, std = calc_mean_std(Subset(stratified_train_set, train_fold_indices))

    train_sampler = torch.utils.data.SubsetRandomSampler(train_fold_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_fold_indices)


    train_loader = torch.utils.data.DataLoader(
        dataset=stratified_train_set,
        batch_size=batch_size,
        sampler=train_sampler,
        worker_init_fn=seed_worker,
        generator=g)

    val_loader = torch.utils.data.DataLoader(
        dataset=stratified_train_set,
        batch_size=batch_size,
        sampler=val_sampler,
        worker_init_fn=seed_worker,
        generator=g)

    model = Net2().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    adam_train_and_validate(model, train_loader, val_loader, criterion, optimizer, mean, std, epochs=num_epochs)
    break

Fold 1/5


/Users/alishihab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/alishihab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([128, 128, 16, 16])
conv4 torch.Size([128, 128, 8, 8])
flatten torch.Size([128, 8192])
fc1 torch.Size([128, 256])
fc2 torch.Size([128, 256])
fc3 torch.Size([128, 10])
input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([128, 128, 16, 16])
conv4 torch.Size([128, 128, 8, 8])
flatten torch.Size([128, 8192])
fc1 torch.Size([128, 256])
fc2 torch.Size([128, 256])
fc3 torch.Size([128, 10])
input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([128, 128, 16, 16])
conv4 torch.Size([128, 128, 8, 8])
flatten torch.Size([128, 8192])
fc1 torch.Size([128, 256])
fc2 torch.Size([128, 256])
fc3 torch.Size([128, 10])
input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([12

In [9]:
# test model
test_loader = torch.utils.data.DataLoader(
    dataset=stratified_test_set,
    batch_size=batch_size,
    worker_init_fn=seed_worker,
    generator=g)

test(model, test_loader, mean, std)

input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([128, 128, 16, 16])
conv4 torch.Size([128, 128, 8, 8])
flatten torch.Size([128, 8192])
fc1 torch.Size([128, 256])
fc2 torch.Size([128, 256])
fc3 torch.Size([128, 10])
input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([128, 128, 16, 16])
conv4 torch.Size([128, 128, 8, 8])
flatten torch.Size([128, 8192])
fc1 torch.Size([128, 256])
fc2 torch.Size([128, 256])
fc3 torch.Size([128, 10])
input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([128, 128, 16, 16])
conv4 torch.Size([128, 128, 8, 8])
flatten torch.Size([128, 8192])
fc1 torch.Size([128, 256])
fc2 torch.Size([128, 256])
fc3 torch.Size([128, 10])
input torch.Size([128, 3, 32, 32])
conv1 torch.Size([128, 64, 32, 32])
conv2 torch.Size([128, 64, 32, 32])
conv3 torch.Size([12

Net2(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU()
  (batch_norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout1): Dropout(p=0.1, inplace=False)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu4): ReLU()
  (batch_norm4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

## Population Based - Genetic Algorithm

## Proposed - Adaptive Baldwinian-Lamarckian Memetic Algorithm
## Adam-guided Adaptive-SL-PSO

### Imports, Preprocessing & Definitions

In [10]:
import operator
import random
from matplotlib import pyplot as plt
import math
from deap import base
from deap import benchmarks
from deap import creator
from deap import tools
from numba import jit, cuda
from numpy import genfromtxt

In [11]:
def neighbour_search(pop, individual, neighbours=10):
    distances = {}
    for i, part in enumerate(pop):
        # print(f"part: {len(part)} individual: {len(individual)}")
        if individual != part:
            distance = np.linalg.norm(np.array(part)-np.array(individual))
            if len(distances) < neighbours:
                distances[i] = distance
            else:
                copy = distances.copy()
                for e in copy.keys():
                    if distances[e] > distance:
                        distances[i] = distance
                        del distances[e]
                        break
    neighbourhood = [pop[i] for i in distances.keys()]
    sort_population(neighbourhood, potential=True)
    return neighbourhood

# Ali's functions for question 3 ----------------------------------------------------------------
def sort_population(population, potential=False):
    if potential:
        population.sort(key=lambda x: x.potential[0])
    else:
        population.sort(key=lambda x: x.fitness.values[0])

In [62]:
# Function to freeze all but the last layer
def freeze_all_but_last(model):
    for name, param in model.named_parameters():
        if 'fc3' not in name:
            param.requires_grad = False

# Extract weights from the last layer
def extract_weights_biases(layer):
    return [p.data.numpy() for p in layer.parameters()]

def generate_particle(dimension):
    part = creator.Particle([random.uniform(-1, 1) for _ in range(dimension)])
    part.speed = [random.uniform(-1, 1) for _ in range(dimension)]
    part.smin = -1
    part.smax = 1
    return part

# Define the fitness function
def evaluate_particle(model, particle, inputs, labels, best=False):
    weights = np.asarray(particle, dtype=np.float32)
    new_weights = torch.from_numpy(weights[:weights_len].reshape(weights_dim))
    new_biases = torch.from_numpy(weights[weights_len:dimension])
    new_weights = new_weights.to(device)
    new_biases = new_biases.to(device)
    model.fc3.weight = torch.nn.Parameter(new_weights)
    model.fc3.bias = torch.nn.Parameter(new_biases)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    inputs, labels = torch.FloatTensor(inputs), torch.Tensor(labels)
    inputs, labels, model = inputs.to(device), labels.to(device), model.to(device)
    optimizer.zero_grad()  # clear gradients for next train

    # fitness
    outputs = model(inputs)  # input and predict based on images
    loss = criterion(outputs, labels)
    fitness = loss.item(),
    
    # potential
    loss.backward()  # backpropagation, compute gradients
    optimizer.step()  # apply gradients
    optimizer.zero_grad()  # clear gradients for next train
    outputs = model(inputs)  # input and predict based on images
    loss = criterion(outputs, labels)
    potential = loss.item(),

    if best:
            # potential turns into fitness
            fitness = potential

            # and next gradient descent becomes the potential
            loss.backward()  # backpropagation, compute gradients
            optimizer.step()  # apply gradients
            optimizer.zero_grad()  # clear gradients for next train
            outputs = model(inputs)  # input and predict based on images
            loss = criterion(outputs, labels)
            potential = loss.item(),

            demonstrator_weights = model.fc3.weight.data.to('cpu').reshape(-1)
            demosntrator_biases = model.fc3.bias.data.to('cpu').reshape(-1)
            new_lamarckian = np.concatenate((demonstrator_weights, demosntrator_biases))
            new_particle = creator.Particle(new_lamarckian)
            new_particle.fitness.values = fitness
            new_particle.potential = potential
            
            return new_particle
    else:
            particle.fitness.values = fitness
            particle.potential = potential
    #return loss.item(),

# social learning in a neighbourhood of size
def behaviour_learning(gamma, gbest, part, pop, mu, epsilon, neighbours=10):
    i = pop.index(part)
    neighbour_pop = neighbour_search(pop, part, neighbours)

    index = None
    for j in range(len(neighbour_pop)-1, -1, -1):
        if neighbour_pop[j].potential[0] <= part.potential[0]:
            index = j
            break


    demonstrator = None

    if (i != 0) & (index is not None):
        if index != 0:
                k = math.floor(random.randrange(0, index))
                demonstrator = neighbour_pop[k]
        else:
                k = index
                demonstrator = neighbour_pop[k]
    else:
        demonstrator = [sum(np.asarray(neighbour_pop)[:,x])/populationSize for x in range(dimension)]
        demonstrator = -1*np.asarray(demonstrator)

    r1 = (random.uniform(0, 1) for _ in range(len(part)))
    r2 = (random.uniform(0, 1) for _ in range(len(part)))
    r3 = (random.uniform(0, 1) for _ in range(len(part)))
    ones = [1] * len(part)
    one_minus_gamma = np.asarray(ones) - gamma

    v_r0 = list(map(operator.mul, r1, part.speed))
    v_r1 = list(map(operator.mul, r2, map(operator.sub, demonstrator, part))) # local best
    v_r2 = list(map(operator.mul,r3, map(operator.mul, [epsilon*x for x in mu], part))) # global best

    sl_speed = list(map(operator.add, v_r1, v_r2))
    exploitation_speed = gamma*np.asarray(gbest)
    exploration_speed = list(map(operator.mul, one_minus_gamma, sl_speed))

    new_speed = list(map(operator.add, v_r0 , map(operator.add, exploitation_speed, exploration_speed)))
    new_part = list(map(operator.add, part, part.speed))

    return new_part, new_speed

In [77]:
def pso_optimize(model, toolbox, pop, inputs, labels, batch):
    interval        = 50
    iterations      = 20
    neighbours = 10
    beta = 0.01
    alpha = 0.5

    m = populationSize + math.floor(dimension/10)
    epsilon = beta * (dimension/populationSize)

    gbest = None
    new_weights = None
    new_biases = None

    # eval current fitness - only on the first run
    if batch == 1:
        for part in pop:
            toolbox.evaluate(model, part, inputs, labels) #actually only one fitness value

        # find the global best - lamarckian search party lead - gradient descent
        sort_population(pop, potential=False)
        gbest = pop[0]

        # sort the the baldwinian search participants by their potential, leave the current global best as the lamarckian search-lead
        sort_population(pop[1:], potential=True)

    # begin the evolution
    #for g in range(iterations):

    # a new Search
    print("-- Search %i --" % batch)

    print(f'current gbest: {gbest.fitness.values[0]}, current worst: {pop[-1].fitness.values[0]}')

    # parameter setting
    mu = [sum(np.asarray(pop)[:,x])/populationSize for x in range(dimension)]
    gamma = min(gbest.fitness.values[0]-gbest.potential[0]/np.std(fits), 1) if len(fits) > 5 else (1-(gbest.fitness.values[0]/pop[-2].fitness.values))
    i = 0

    # evolve the local-search-groups via SL-PSO algorithm
    for part in pop[1:]:
        i = i + 1
        learn_prob = (1 - (i-1)/m)**(alpha*math.log(math.ceil(dimension/m)))
        if random.random() < learn_prob:
            part[:], part.speed = toolbox.learn(gamma, gbest, part, pop[1:], mu, epsilon, neighbours)
            toolbox.evaluate(model, part, inputs, labels) # evaluate fitness & potential


    # evolve the lamarckian lead
    pop[0] = toolbox.evaluate(model, pop[0], inputs, labels, best=True) #actually only one fitness value

    # find the global best - lamarckian search party lead - gradient descent
    sort_population(pop, potential=False)
    gbest = pop[0]

    # sort the the baldwinian search participants by their potential, leave the current global best as the lamarckian search-lead
    sort_population(pop[1:], potential=True)

    print(f'gbest after evolution: {gbest.fitness.values[0]}, worst after evolution: {pop[-1].fitness.values[0]}')

    # set weights to best individual
    weights = np.asarray(gbest)
    new_weights = torch.from_numpy(weights[:weights_len].reshape(weights_dim)).to(device)
    new_biases = torch.from_numpy(weights[weights_len:dimension]).to(device)

    # Gather all the fitnesses in one list and print the stats
    # print every interval
    fits.append(gbest.fitness.values[0])
    if batch%interval==0: # interval
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5


        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        plt.plot(fits)
    return new_weights, new_biases, pop

In [78]:
# function for training and evaluating the model
def memetic_train_and_validate(model, toolbox, pop, train_loader, test_loader, criterion, optimizer, mean, std, epochs=30):
    model.to(device)
    model.eval() # turn off dropout layers

    # early stopping parameters
    early_stopping_patience = 3  # number of epochs to wait for improvement before stopping
    early_stopping_counter = 0    # counter for epochs without improvement
    best_accuracy = 0             # track the best accuracy
    loss_curve = []  # track loss curve
    accuracy_curve = []  # track accuracy curve

    # train
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        i = 0

        for inputs, train_load_labels in train_loader:
            inputs = train_transform(inputs, mean, std)
            #inputs, train_load_labels = inputs.to(device), train_load_labels.to(device)
            i += 1
            new_weights, new_biases, pop = pso_optimize(model, toolbox, pop, inputs, train_load_labels, i)
            model.fc3.weight = torch.nn.Parameter(new_weights)
            model.fc3.biases = torch.nn.Parameter(new_biases)
            inputs, train_load_labels = inputs.to(device), train_load_labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, train_load_labels)
            loss_curve.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            running_loss += loss.item()
            total += train_load_labels.size(0)
            correct += (predicted == train_load_labels).sum().item()
            #if i % 20 == 0:
            batch_accuracy = 100 * correct / total
            accuracy_curve.append(batch_accuracy)
            print(f'{i}th Batch Loss: {loss.item():.4f} Batch Accuracy: {batch_accuracy:.4f}')

        epoch_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{epochs}] Loss: {loss.item():.4f} Epoch Accuracy: {epoch_accuracy:.4f}')

        model.eval()
        correct = 0
        total = 0

        # validate
        with torch.no_grad():
            for inputs, test_load_labels in test_loader:
                #inputs = normalize(inputs, mean, std)
                inputs, test_load_labels = inputs.to(device), test_load_labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += test_load_labels.size(0)
                correct += (predicted == test_load_labels).sum().item()

        validation_accuracy = 100 * correct / total

        # check if the current validation accuracy is better than the best recorded accuracy
        if validation_accuracy > best_accuracy:
            best_accuracy = validation_accuracy
            early_stopping_counter = 0  # Reset the counter
            # save the model checkpoint
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, filename=f"best_model_epoch_{epoch+1}.pth.tar")
        else:
            early_stopping_counter += 1

        print(early_stopping_counter)
        # check if early stopping should be triggered
        if early_stopping_counter >= early_stopping_patience:
            print("Early stopping triggered")
            break

        print(f'Validation Accuracy: {validation_accuracy:.2f}%')
    return model, loss_curve, accuracy_curve

In [75]:
best_checkpoint = torch.load("best_model_epoch_19.pth.tar")  # Replace X with the epoch number
model = Net2()
model.load_state_dict(best_checkpoint['state_dict'])

<All keys matched successfully>

In [79]:
fc3_weights = model.fc3.weight.data
weights_dim = fc3_weights.shape
weights_len = len(fc3_weights.reshape(-1))
fc3_bias = model.fc3.bias.data
bias_dim = fc3_bias.shape
bias_len = len(fc3_bias.reshape(-1))
populationSize  = 100
dimension = weights_len + bias_len

# Freeze all layers except the last
freeze_all_but_last(model)
if "Particle" in dir(creator) or "FitnessMin" in dir(creator):
    del creator.Particle, creator.FitnessMin


# DEAP inits
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Particle", list, fitness=creator.FitnessMin, speed=list,
                smin=None, smax=None, best=None, potential=None)

toolbox = base.Toolbox()
toolbox.register("particle", generate_particle, dimension)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("evaluate", evaluate_particle)
toolbox.register("learn", behaviour_learning)

# Model hyperparameters
dropout_prob = 0.30
num_epochs = 20
batch_size = 128

loss_curve = None
accuracy_curve = None

# call function
# Main loop for k-fold cross-validation
for fold, (train_fold_indices, val_fold_indices) in enumerate(skf.split(train_idx, stratified_train_labels)):
    print(f'Fold {fold + 1}/{num_folds}')
    mean, std = calc_mean_std(Subset(stratified_train_set, train_fold_indices))

    train_sampler = torch.utils.data.SubsetRandomSampler(train_fold_indices)
    val_sampler = torch.utils.data.SubsetRandomSampler(val_fold_indices)

    # create an initial population of individuals
    pop = toolbox.population(n=populationSize)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)


    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    fits = []


    train_loader = torch.utils.data.DataLoader(
        dataset=stratified_train_set,
        batch_size=batch_size,
        sampler=train_sampler,
        worker_init_fn=seed_worker,
        generator=g)

    val_loader = torch.utils.data.DataLoader(
        dataset=stratified_train_set,
        batch_size=batch_size,
        sampler=val_sampler,
        worker_init_fn=seed_worker,
        generator=g)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    model, loss_curve, accuracy_curve = memetic_train_and_validate(model, toolbox, pop, train_loader, val_loader, criterion, optimizer, mean, std, epochs=1)
    break

Fold 1/5


/Users/alishihab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/alishihab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
